### Import libraries

In [1]:
%matplotlib inline

import csv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas.io.json import json_normalize

import matplotlib
import matplotlib.pyplot as plt
import plotly.plotly as py
import plotly.tools as tls
from plotly.graph_objs import *

import time

from datetime import datetime
from dateutil import tz

from lxml import objectify

from bibliopixel import *
from bibliopixel.drivers.LPD8806 import *
from bibliopixel import LEDStrip
import bibliopixel.colors as colors

In [20]:
## read data
df_total = pd.read_pickle('All_meters_concat')

df_new = pd.read_pickle('Dunne_Daily_Average')

time_stamp
2015-01-16 01:00:00    54.252
2015-01-16 01:30:00    55.044
2015-01-16 02:00:00    55.908
2015-01-16 02:30:00    55.476
2015-01-16 03:00:00    55.764
2015-01-16 03:30:00    55.683
2015-01-16 04:00:00    56.259
2015-01-16 04:30:00    55.755
2015-01-16 05:00:00    55.611
2015-01-16 05:30:00    56.250
2015-01-16 06:00:00    57.186
2015-01-16 06:30:00    56.331
2015-01-16 07:00:00    57.987
2015-01-16 07:30:00    59.724
2015-01-16 08:00:00    62.784
2015-01-16 08:30:00    61.560
2015-01-16 09:00:00    62.703
2015-01-16 09:30:00    70.839
2015-01-16 10:00:00    67.671
2015-01-16 10:30:00    65.592
2015-01-16 11:00:00    62.712
2015-01-16 11:30:00    59.040
2015-01-16 12:00:00    52.056
2015-01-16 12:30:00    50.247
2015-01-16 13:00:00    49.752
2015-01-16 13:30:00    50.895
2015-01-16 14:00:00    53.559
2015-01-16 14:30:00    51.696
2015-01-16 15:00:00    51.255
2015-01-16 15:30:00    49.662
                        ...  
2016-02-16 04:00:00    32.525
2016-02-16 04:30:00    33.413

### Generating color scale

In [12]:
red = 0
green = 255
stepSize = 50
color_gn_rd = []
color_gn_rd.append((red, green, 0)) ## the lights are GRB format

In [13]:
while(red < 255): ## start with green and increase red
    red += stepSize;
    if(red > 245):
        red = 255; 
    color_gn_rd.append((red, green, 0))

while(green > 0): ## start with red + green and decrease green
    green -= stepSize;
    if(green < 6):
        green = 0; 
    color_gn_rd.append((red, green, 0)); 
    
total_colors = len(color_gn_rd)
total_colors

11

### Setting up LEDs

In [14]:
LedsPerSide = 10
numLeds= LedsPerSide*4*2 ##x/side * 4 sides * 2 levels
driver=DriverLPD8806(numLeds, ChannelOrder.BRG)
led=LEDStrip(driver)

ERROR - spi_driver_base - Unable to import spidev. Please install. pip install spidev
ERROR:BiblioPixel:Unable to import spidev. Please install. pip install spidev


ImportError: Unable to import spidev. Please install. pip install spidev

### Defining Flashing modes

In [15]:
def led_set(start_position, numLEDs, color): ## Fills the colors
    led.fill(color, start=start_position,end=start_position+numLEDs)
    led.update()
    #print color
    return

def pulse(start_position, numLEDs, color):
    ## Step-up intensity by 10% increments, then step down by the same every 0.1 seconds. total time = 4 sec
    
    intensity = np.arange(0,1.1,0.1) 

    for i in intensity:
        color_new = (int(color[0]*i),int(color[1]*i),int(color[2]*i)) 
        # There is probably a more elegant way to do this.. 
        led.fill(color_new, start=start_position,end=start_position+numLEDs)
        led.update()
        time.sleep(0.2)

    for i in reversed(intensity):
        color_new = (color[0]*i,color[1]*i,color[3]*i) 
        led.fill(color_new, start=start_position,end=start_position+numLEDs)
        led.update()
        time.sleep(0.2)        

In [16]:
print ('Starting Display')
print ('Press \'control + c\' to stop' )


Starting Display
Press 'control + c' to stop


### Calculate Average Data

In [17]:
#Averaged by Month

df_total['Month'] = pd.DatetimeIndex(df_total['time_stamp']).month
df_Monthlygroup = df_total.groupby('Month')
df_MonthlyAverage = df_Monthlygroup['value'].mean()

#Averaged by Day

df_total['Date'] = pd.DatetimeIndex(df_total['time_stamp']).date
df_Dailygroup = df_total.groupby('Date')
df_DailyAverage = df_Dailygroup['value'].mean()
df_RollingMean = pd.rolling_mean(df_DailyAverage, 30)


### Displaying Daily Values

In [18]:
min_value = df_DailyAverage.min()
max_value = df_DailyAverage.max()
ScalingSteps = (max_value - min_value)/(total_colors - 1)
ScalingSteps


2.4753895833333335

In [55]:
for item in df_DailyAverage.iteritems():
    print 'Date: ', item[0]
    print 'Average use: ', item [1]
    color_index = int(item[1]/ScalingSteps)
    
    # print color_index
    color = color_gn_rd[color_index]
    led_set(0, 80, color)

Date:  2015-09-01
Average use:  7.35225
2
(100, 255, 0)
Date:  2015-09-02
Average use:  7.73325
3
(150, 255, 0)
Date:  2015-09-03
Average use:  7.16625
2
(100, 255, 0)
Date:  2015-09-04
Average use:  6.774
2
(100, 255, 0)
Date:  2015-09-05
Average use:  5.792625
2
(100, 255, 0)
Date:  2015-09-06
Average use:  6.916125
2
(100, 255, 0)
Date:  2015-09-07
Average use:  7.066875
2
(100, 255, 0)
Date:  2015-09-08
Average use:  8.298375
3
(150, 255, 0)
Date:  2015-09-09
Average use:  6.84675
2
(100, 255, 0)
Date:  2015-09-10
Average use:  5.422125
2
(100, 255, 0)
Date:  2015-09-11
Average use:  4.605
1
(50, 255, 0)
Date:  2015-09-12
Average use:  2.773125
1
(50, 255, 0)
Date:  2015-09-13
Average use:  3.153
1
(50, 255, 0)
Date:  2015-09-14
Average use:  4.588875
1
(50, 255, 0)
Date:  2015-09-15
Average use:  5.67225
2
(100, 255, 0)
Date:  2015-09-16
Average use:  6.43125
2
(100, 255, 0)
Date:  2015-09-17
Average use:  7.36725
2
(100, 255, 0)
Date:  2015-09-18
Average use:  6.587625
2
(100, 25

### Displaying Daily Values on Bottom with last 30 days average on Top

In [ ]:
min_value = df_DailyAverage.min()
max_value = df_DailyAverage.max()
ScalingSteps = (max_value - min_value)/(total_colors - 1)
ScalingSteps

In [64]:
count = 0
for item in df_DailyAverage.iteritems():
    print 'Date: ', item[0]
    print 'Average use: ', item [1]
    color_index = int(item[1]/ScalingSteps)
    
    # print color_index
    color = color_gn_rd[color_index]
    led_set(0, 40, color)
    
    print 'Last 30 days average: ', df_RollingMean[count]
    try:
        color_index = int(df_RollingMean[count]/ScalingSteps)
        color = color_gn_rd[color_index]
        led_set(40, 80, color)

    except:
        color = (0,0,0)
        led_set(40, 80, color)
    
    count = count + 1
        
    

Date:  2015-09-01
Average use:  7.35225
(100, 255, 0)
Last 30 days average:  nan
(0, 0, 0)
Date:  2015-09-02
Average use:  7.73325
(150, 255, 0)
Last 30 days average:  nan
(0, 0, 0)
Date:  2015-09-03
Average use:  7.16625
(100, 255, 0)
Last 30 days average:  nan
(0, 0, 0)
Date:  2015-09-04
Average use:  6.774
(100, 255, 0)
Last 30 days average:  nan
(0, 0, 0)
Date:  2015-09-05
Average use:  5.792625
(100, 255, 0)
Last 30 days average:  nan
(0, 0, 0)
Date:  2015-09-06
Average use:  6.916125
(100, 255, 0)
Last 30 days average:  nan
(0, 0, 0)
Date:  2015-09-07
Average use:  7.066875
(100, 255, 0)
Last 30 days average:  nan
(0, 0, 0)
Date:  2015-09-08
Average use:  8.298375
(150, 255, 0)
Last 30 days average:  nan
(0, 0, 0)
Date:  2015-09-09
Average use:  6.84675
(100, 255, 0)
Last 30 days average:  nan
(0, 0, 0)
Date:  2015-09-10
Average use:  5.422125
(100, 255, 0)
Last 30 days average:  nan
(0, 0, 0)
Date:  2015-09-11
Average use:  4.605
(50, 255, 0)
Last 30 days average:  nan
(0, 0, 0)